<br>

# Introdução

Atualiza o valor da UFESP


In [ ]:
import pandas as pd
#from pathlib import Path
from ufesp.paths import *

In [ ]:
# Paths
#project_path = Path().resolve().absolute().parent
#data_path = project_path / 'ufesp' / 'data'

project_path

In [ ]:
URL = 'https://legislacao.fazenda.sp.gov.br/Paginas/ValoresDaUFESP.aspx'
dfs = pd.read_html(URL)


In [ ]:
df = dfs[2]
df


In [ ]:
df.loc[:, 'VALOR EM R$'] = df['VALOR EM R$'].replace(',', '', regex=True)
df.loc[:, 'VALOR EM R$'] = df['VALOR EM R$'].replace('\u200b', '', regex=True)
df.loc[:, 'VALOR EM R$'] = df['VALOR EM R$'].astype(float)
df.loc[:, 'VALOR EM R$'] = df['VALOR EM R$'] / 100
df


In [ ]:
df.loc[:, 'PERÍODO'] = df['PERÍODO'].replace('de ', '', regex=True)
df.loc[:, 'PERÍODO'] = df['PERÍODO'].replace(' a ', ' A ', regex=True)

df.loc[:, 'PERÍODO'] = df['PERÍODO'].replace('\u200b', '', regex=True)
df.loc[:, 'PERÍODO'] = df['PERÍODO'].replace('\xa0', ' ', regex=True)
df[['data_inicio', 'data_fim']] = df['PERÍODO'].str.split('A', 1, expand=True)

df.loc[:, 'data_inicio'] = df.loc[:, 'data_inicio'].str.strip()
df.loc[:, 'data_fim'] = df.loc[:, 'data_fim'].str.strip()

df.loc[:, 'data_inicio'] = pd.to_datetime(df['data_inicio'], format='%d/%m/%Y')
df.loc[:, 'data_fim'] = pd.to_datetime(df['data_fim'], format='%d/%m/%Y')

df.loc[:, 'ano_mes'] = pd.to_datetime(df['data_inicio']).dt.to_period('M')

df.rename({'VALOR EM R$': 'valor', 'BASE LEGAL': 'base_legal'},
          axis=1, inplace=True)

df.loc[:, 'base_legal'] = df['base_legal'].replace('\u200b', '', regex=True)
df.loc[:, 'base_legal'] = df.loc[:, 'base_legal'].str.strip()

df = df[['data_inicio', 'data_fim', 'ano_mes', 'valor', 'base_legal']]
df


In [ ]:

# Save
df.to_csv(
    data_path / 'ufesp.csv',
    index=False,
    decimal=',',
)